In [1]:
from cng.utils import *

import ibis
from ibis import _

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
set_secrets(con)
duckdb_install_h3()

parquet = "https://minio.carlboettiger.info/public-data/social-vulnerability/2022/SVI2022_US_tract.parquet" # faster with local
con.raw_sql(f"CREATE  OR REPLACE VIEW svi AS SELECT FIPS, Shape AS geom FROM '{parquet}'")


In [ ]:
zoom = 11

con.sql(f'''
WITH t1 AS (
  SELECT FIPS, ST_Dump(geom) AS geom 
  FROM svi
) 
SELECT FIPS, 
       h3_polygon_wkt_to_cells_string(UNNEST(geom).geom, {zoom}) AS h{zoom}
FROM t1
''').to_parquet(f"s3://public-social-vulnerability/2022-tracts-h3-z{zoom}.parquet")